In [ ]:
import pandas as pd 
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://archive.ubuntu.co

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-20 03:52:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.23MB/s    in 0.2s    

2022-05-20 03:52:13 (6.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://spotifyfinal-bucket.s3.amazonaws.com/spotify_dataset.csv"
spark.sparkContext.addFile(url)
raw_df = spark.read.csv(SparkFiles.get("spotify_dataset.csv"), sep=",", header=True, inferSchema=True)

raw_df.head()

Row(track='Jealous Kind Of Fella', artist='Garland Green', uri='spotify:track:1dtKN6wwlolkM8XZy2y9C1', danceability='0.417', energy='0.62', key='3', loudness='-7.727', mode=1.0, speechiness=0.0403, acousticness=0.49, instrumentalness=0.0, liveness=0.0779, valence=0.845, tempo=185.655, duration_ms=173533.0, time_signature=3.0, chorus_hit=32.94975, sections=9.0, popularity=1.0, decade='60s')

In [ ]:
raw_df.show()

+--------------------+--------------------+--------------------+--------------------+------+-----+--------+-------+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+--------+----------+------+
|               track|              artist|                 uri|        danceability|energy|  key|loudness|   mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|chorus_hit|sections|popularity|decade|
+--------------------+--------------------+--------------------+--------------------+------+-----+--------+-------+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+--------+----------+------+
|Jealous Kind Of F...|       Garland Green|spotify:track:1dt...|               0.417|  0.62|    3|  -7.727|    1.0|     0.0403|        0.49|             0.0|  0.0779|  0.845|185.655|   173533.0|           3.0|  32.94975|     9.0|       1.0|   60s|
|       

Cleaning data

In [ ]:
clean1df = raw_df.drop_duplicates(subset=['uri'])

clean1df.show()

+--------------------+--------------------+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+--------+----------+------+
|               track|              artist|                 uri|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|chorus_hit|sections|popularity|decade|
+--------------------+--------------------+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+--------+----------+------+
|           Spotlight|     Jennifer Hudson|spotify:track:02Q...|       0.707| 0.724| 11|  -3.887| 0.0|      0.051|      0.0448|             0.0|   0.072|  0.659|108.984|   250107.0|           4.0|  61.48974|    10.0|       1.0|   00s|
|         Tres Pontas|   Milton Nascimento|spotify:track:03v

In [ ]:
clean2df = clean1df.drop("uri","artist", "track")
clean2df.show()

+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+--------+----------+------+
|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|chorus_hit|sections|popularity|decade|
+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+--------+----------+------+
|       0.707| 0.724| 11|  -3.887| 0.0|      0.051|      0.0448|             0.0|   0.072|  0.659|108.984|   250107.0|           4.0|  61.48974|    10.0|       1.0|   00s|
|       0.348| 0.524|  7| -14.179| 1.0|     0.0417|       0.657|         6.72E-5|  0.0931|  0.348| 86.311|   160173.0|           5.0|  84.14468|     5.0|       0.0|   60s|
|       0.895| 0.491| 10| -10.261| 0.0|      0.246|      0.0144|         9.88E-4|   0.143|  0.249| 83.505|   227727.0|           4.0|  35.25

In [ ]:
Info_df = clean1df.drop("danceability","energy", "key", "loudness", "mode", "speechiness", "acousticness", 
                              "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature",
                              "chorus_hit", "sections")

Info_df.show()

+--------------------+--------------------+--------------------+----------+------+
|               track|              artist|                 uri|popularity|decade|
+--------------------+--------------------+--------------------+----------+------+
|           Spotlight|     Jennifer Hudson|spotify:track:02Q...|       1.0|   00s|
|         Tres Pontas|   Milton Nascimento|spotify:track:03v...|       0.0|   60s|
|         No Shopping|French Montana Fe...|spotify:track:0Di...|       1.0|   10s|
|             Burarum|               Phazm|spotify:track:0KB...|       0.0|   00s|
|             Respect|    The Real Roxanne|spotify:track:0LL...|       0.0|   80s|
|      Hollaback Girl|        Gwen Stefani|spotify:track:0Lz...|       1.0|   00s|
|     Keep On Running|The Spencer Davis...|spotify:track:0XX...|       1.0|   60s|
|       To Love Again|         Alan Pasqua|spotify:track:0YK...|       0.0|   90s|
|  Prove It All Night|   Bruce Springsteen|spotify:track:0Ye...|       1.0|   70s|
|Win

In [ ]:
Characteristics_df = clean1df.drop("track", "artist", "popularity", "decade")

Characteristics_df.show()

+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+--------+
|                 uri|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|chorus_hit|sections|
+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+--------+
|spotify:track:02Q...|       0.707| 0.724| 11|  -3.887| 0.0|      0.051|      0.0448|             0.0|   0.072|  0.659|108.984|   250107.0|           4.0|  61.48974|    10.0|
|spotify:track:03v...|       0.348| 0.524|  7| -14.179| 1.0|     0.0417|       0.657|         6.72E-5|  0.0931|  0.348| 86.311|   160173.0|           5.0|  84.14468|     5.0|
|spotify:track:0Di...|       0.895| 0.491| 10| -10.261| 0.0|      0.246|      0.0144|         9.88E-4|   0.143|  0.249| 83.50

In [ ]:
Characteristics_df.dtypes

[('uri', 'string'),
 ('danceability', 'string'),
 ('energy', 'string'),
 ('key', 'string'),
 ('loudness', 'string'),
 ('mode', 'double'),
 ('speechiness', 'double'),
 ('acousticness', 'double'),
 ('instrumentalness', 'double'),
 ('liveness', 'double'),
 ('valence', 'double'),
 ('tempo', 'double'),
 ('duration_ms', 'double'),
 ('time_signature', 'double'),
 ('chorus_hit', 'double'),
 ('sections', 'double')]

In [ ]:
Characteristics_df = Characteristics_df.withColumn("danceability",Characteristics_df.danceability.cast('double'))
Characteristics_df = Characteristics_df.withColumn("energy",Characteristics_df.energy.cast('double'))
Characteristics_df = Characteristics_df.withColumn("key",Characteristics_df.key.cast('double'))
Characteristics_df = Characteristics_df.withColumn("loudness",Characteristics_df.loudness.cast('double'))

Characteristics_df.dtypes

[('uri', 'string'),
 ('danceability', 'double'),
 ('energy', 'double'),
 ('key', 'double'),
 ('loudness', 'double'),
 ('mode', 'double'),
 ('speechiness', 'double'),
 ('acousticness', 'double'),
 ('instrumentalness', 'double'),
 ('liveness', 'double'),
 ('valence', 'double'),
 ('tempo', 'double'),
 ('duration_ms', 'double'),
 ('time_signature', 'double'),
 ('chorus_hit', 'double'),
 ('sections', 'double')]

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://spotifyaws.cxmgxnqnrkea.us-east-2.rds.amazonaws.com:5432/spotifydata"
config = {"user":"postgres", 
          "password": "$potify!", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write Info to table in RDS
Info_df.write.jdbc(url=jdbc_url, table='Info', mode=mode, properties=config)

In [ ]:
# Write Info to table in RDS
Characteristics_df.write.jdbc(url=jdbc_url, table='characteristics', mode=mode, properties=config)